<a href="https://colab.research.google.com/github/shitizz24/Time-Series-Forecasting/blob/master/Support_Vector_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
import numpy as np
# import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

In [63]:
from numpy.random import seed
seed(1)

In [64]:
dataset=pd.read_csv("Data/testset.csv")
dataset

,datetime_utc,_conds,_dewptm,_fog,_hail,_heatindexm,_hum,_precipm,_pressurem,_rain,_snow,_tempm,_thunder,_tornado,_vism,_wdird,_wdire,_wgustm,_windchillm,_wspdm
0,19961101-11:00,Smoke,9.0,0,0,NaN,27.0,NaN,1010.0,0,0,30.0,0,0,5.00,280.0,West,NaN,NaN,7.4
1,19961101-12:00,Smoke,10.0,0,0,NaN,32.0,NaN,-9999.0,0,0,28.0,0,0,NaN,0.0,North,NaN,NaN,NaN
2,19961101-13:00,Smoke,11.0,0,0,NaN,44.0,NaN,-9999.0,0,0,24.0,0,0,NaN,0.0,North,NaN,NaN,NaN
3,19961101-14:00,Smoke,10.0,0,0,NaN,41.0,NaN,1010.0,0,0,24.0,0,0,2.00,0.0,North,NaN,NaN,NaN
4,19961101-16:00,Smoke,11.0,0,0,NaN,47.0,NaN,1011.0,0,0,23.0,0,0,1.20,0.0,North,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99565,20161231-12:00,Partly Cloudy,15.0,0,0,NaN,77.0,NaN,1015.0,0,0,18.0,0,0,0.50,110.0,ESE,NaN,NaN,7.4
99566,20161231-13:00,Partly Cloudy,16.0,0,0,NaN,100.0,NaN,1014.0,0,0,16.0,0,0,0.80,90.0,East,NaN,NaN,3.7
99567,20161231-15:00,NaN,13.0,0,0,NaN,NaN,NaN,1016.0,0,0,NaN,0,0,0.05,10.0,North,NaN,NaN,59.3
99568,20161231-18:00,Patches of Fog,12.0,1,0,NaN,100.0,NaN,1016.0,0,0,12.0,0,0,0.50,NaN,NaN,NaN,NaN,0.0


In [65]:
dataset.columns=dataset.columns.str.replace("_","")
dataset.columns=dataset.columns.str.replace(" ","")
dataset.columns

Index(['datetimeutc', 'conds', 'dewptm', 'fog', 'hail', 'heatindexm', 'hum',
       'precipm', 'pressurem', 'rain', 'snow', 'tempm', 'thunder', 'tornado',
       'vism', 'wdird', 'wdire', 'wgustm', 'windchillm', 'wspdm'],
      dtype='object')

In [66]:
dataset.drop(columns=["conds","dewptm","fog","hail","heatindexm","precipm","rain","snow","thunder","tornado","vism","wdird","wdire","wgustm","windchillm"],inplace=True,axis=1)
dataset

,datetimeutc,hum,pressurem,tempm,wspdm
0,19961101-11:00,27.0,1010.0,30.0,7.4
1,19961101-12:00,32.0,-9999.0,28.0,NaN
2,19961101-13:00,44.0,-9999.0,24.0,NaN
3,19961101-14:00,41.0,1010.0,24.0,NaN
4,19961101-16:00,47.0,1011.0,23.0,0.0
...,...,...,...,...,...
99565,20161231-12:00,77.0,1015.0,18.0,7.4
99566,20161231-13:00,100.0,1014.0,16.0,3.7
99567,20161231-15:00,NaN,1016.0,NaN,59.3
99568,20161231-18:00,100.0,1016.0,12.0,0.0


In [67]:
dataset['pressurem'].replace('-9999.0',np.nan)
dataset.replace(0,np.nan)
dataset.isnull().sum()

datetimeutc       0
hum             756
pressurem       232
tempm           672
wspdm          2358
dtype: int64

In [68]:
dataset

,datetimeutc,hum,pressurem,tempm,wspdm
0,19961101-11:00,27.0,1010.0,30.0,7.4
1,19961101-12:00,32.0,-9999.0,28.0,NaN
2,19961101-13:00,44.0,-9999.0,24.0,NaN
3,19961101-14:00,41.0,1010.0,24.0,NaN
4,19961101-16:00,47.0,1011.0,23.0,0.0
...,...,...,...,...,...
99565,20161231-12:00,77.0,1015.0,18.0,7.4
99566,20161231-13:00,100.0,1014.0,16.0,3.7
99567,20161231-15:00,NaN,1016.0,NaN,59.3
99568,20161231-18:00,100.0,1016.0,12.0,0.0


In [69]:
hum_median=dataset["hum"].median()
dataset["hum"].fillna(hum_median,inplace=True)
pressurem_median=dataset["pressurem"].median()
dataset["pressurem"].fillna(pressurem_median,inplace=True)
wspdm_median=dataset["wspdm"].median()
dataset["wspdm"].fillna(wspdm_median,inplace=True)
tempm_median=dataset["tempm"].median()
dataset["tempm"].fillna(tempm_median,inplace=True)

In [70]:
dataset.isnull().sum()

datetimeutc    0
hum            0
pressurem      0
tempm          0
wspdm          0
dtype: int64

In [71]:
dataset

,datetimeutc,hum,pressurem,tempm,wspdm
0,19961101-11:00,27.0,1010.0,30.0,7.4
1,19961101-12:00,32.0,-9999.0,28.0,7.4
2,19961101-13:00,44.0,-9999.0,24.0,7.4
3,19961101-14:00,41.0,1010.0,24.0,7.4
4,19961101-16:00,47.0,1011.0,23.0,0.0
...,...,...,...,...,...
99565,20161231-12:00,77.0,1015.0,18.0,7.4
99566,20161231-13:00,100.0,1014.0,16.0,3.7
99567,20161231-15:00,58.0,1016.0,27.0,59.3
99568,20161231-18:00,100.0,1016.0,12.0,0.0


In [72]:
dataset["datetimeutc"]=pd.to_datetime(dataset["datetimeutc"])

In [73]:
dataset.set_index("datetimeutc",inplace=True)

In [74]:
dataset

,hum,pressurem,tempm,wspdm
datetimeutc,,,,
1996-11-01 11:00:00,27.0,1010.0,30.0,7.4
1996-11-01 12:00:00,32.0,-9999.0,28.0,7.4
1996-11-01 13:00:00,44.0,-9999.0,24.0,7.4
1996-11-01 14:00:00,41.0,1010.0,24.0,7.4
1996-11-01 16:00:00,47.0,1011.0,23.0,0.0
...,...,...,...,...
2016-12-31 12:00:00,77.0,1015.0,18.0,7.4
2016-12-31 13:00:00,100.0,1014.0,16.0,3.7
2016-12-31 15:00:00,58.0,1016.0,27.0,59.3


In [75]:
dataset=dataset.resample("D").mean()
dataset

,hum,pressurem,tempm,wspdm
datetimeutc,,,,
1996-11-01,52.916667,-2659.666667,22.333333,6.166667
1996-11-02,48.625000,1009.833333,22.916667,7.950000
1996-11-03,55.958333,1010.500000,21.791667,5.020833
1996-11-04,48.055556,1011.333333,22.722222,2.266667
1996-11-05,29.400000,1011.800000,27.800000,10.020000
...,...,...,...,...
2016-12-27,67.550000,1017.200000,16.850000,8.335000
2016-12-28,68.043478,1015.565217,17.217391,3.547826
2016-12-29,87.857143,1016.904762,15.238095,6.000000


In [76]:
def extract_year(value):
  value=value.strftime('%Y-%m-%d')
  return (value[0:4])


def extract_month(value):
    value=value.strftime('%Y-%m-%d')
    return (value[4:6])

def extract_date(value):
    value=value.strftime('%Y-%m-%d')
    return (value[6:8])

In [77]:
dataset.reset_index(inplace=True)
dataset


,datetimeutc,hum,pressurem,tempm,wspdm
0,1996-11-01,52.916667,-2659.666667,22.333333,6.166667
1,1996-11-02,48.625000,1009.833333,22.916667,7.950000
2,1996-11-03,55.958333,1010.500000,21.791667,5.020833
3,1996-11-04,48.055556,1011.333333,22.722222,2.266667
4,1996-11-05,29.400000,1011.800000,27.800000,10.020000
...,...,...,...,...,...
7361,2016-12-27,67.550000,1017.200000,16.850000,8.335000
7362,2016-12-28,68.043478,1015.565217,17.217391,3.547826
7363,2016-12-29,87.857143,1016.904762,15.238095,6.000000
7364,2016-12-30,89.666667,1017.904762,14.095238,6.266667


In [78]:
dataset['day'] = dataset['datetimeutc'].dt.day
dataset['month'] = dataset['datetimeutc'].dt.month
dataset['year'] = dataset['datetimeutc'].dt.year

In [79]:
dataset.isnull().sum()

datetimeutc      0
hum            141
pressurem      141
tempm          141
wspdm          141
day              0
month            0
year             0
dtype: int64

In [80]:
dataset.fillna(dataset.mean(),inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  """Entry point for launching an IPython kernel.


In [81]:
dataset.isnull().sum()

datetimeutc    0
hum            0
pressurem      0
tempm          0
wspdm          0
day            0
month          0
year           0
dtype: int64

In [82]:
dataset_svr=dataset.drop(["datetimeutc"],axis=1)
dataset_svr

,hum,pressurem,tempm,wspdm,day,month,year
0,52.916667,-2659.666667,22.333333,6.166667,1,11,1996
1,48.625000,1009.833333,22.916667,7.950000,2,11,1996
2,55.958333,1010.500000,21.791667,5.020833,3,11,1996
3,48.055556,1011.333333,22.722222,2.266667,4,11,1996
4,29.400000,1011.800000,27.800000,10.020000,5,11,1996
...,...,...,...,...,...,...,...
7361,67.550000,1017.200000,16.850000,8.335000,27,12,2016
7362,68.043478,1015.565217,17.217391,3.547826,28,12,2016
7363,87.857143,1016.904762,15.238095,6.000000,29,12,2016
7364,89.666667,1017.904762,14.095238,6.266667,30,12,2016


In [83]:
y=dataset_svr["tempm"]
y

0       22.333333
1       22.916667
2       21.791667
3       22.722222
4       27.800000
          ...    
7361    16.850000
7362    17.217391
7363    15.238095
7364    14.095238
7365    15.650000
Name: tempm, Length: 7366, dtype: float64

In [84]:
dataset_svr.drop("tempm",inplace=True,axis=1)

In [85]:
dataset_svr.columns

Index(['hum', 'pressurem', 'wspdm', 'day', 'month', 'year'], dtype='object')

In [86]:
X=dataset_svr.iloc[:,:].values
X.shape

(7366, 6)

In [87]:
y=y.values
y.shape

(7366,)

In [88]:
from sklearn.model_selection import train_test_split

In [89]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.10,random_state=0)

In [90]:
print("Shape of X_train", X_train.shape)
print("Shape of X_test", X_test.shape)
print("Shape of y_train", y_train.shape)
print("Shape of y_test", y_test.shape)

Shape of X_train (6629, 6)
Shape of X_test (737, 6)
Shape of y_train (6629,)
Shape of y_test (737,)


In [91]:
from sklearn.preprocessing import StandardScaler
sc_X=StandardScaler()
sc_Y=StandardScaler()
X_train=sc_X.fit_transform(X_train)
y_train=np.array(y_train).reshape(-1,1)
y_train=sc_Y.fit_transform(y_train)
# y_train=np.array(y_train).reshape(-1,1)

In [92]:
from sklearn.svm import SVR
regressor = SVR(kernel = 'rbf')

In [93]:
regressor.fit(X_train,y_train.ravel())

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [94]:
y_pred=regressor.predict(X_test)

In [95]:
y_pred=sc_Y.inverse_transform(y_pred)
y_pred

array([19.84449357, 19.84449357, 19.84449357, 19.84449357, 19.84449357,
       19.84449357, 19.84449357, 19.84449357, 19.84449357, 19.84449357,
       19.84449357, 19.84449357, 19.84449357, 19.84449357, 19.84449357,
       19.84449357, 19.84449357, 19.84449357, 19.84449357, 19.84449357,
       19.84449357, 19.84449357, 19.84449357, 19.84449357, 19.84449357,
       19.84449357, 19.84449357, 19.84449357, 19.84449357, 19.84449357,
       19.84449357, 19.84449357, 19.84449357, 19.84449357, 19.84449357,
       19.84449357, 19.84449357, 19.84449357, 19.84449357, 19.84449357,
       19.84449357, 19.84449357, 19.84449357, 19.84449357, 19.84449357,
       19.84449357, 19.84449357, 19.84449357, 19.84449357, 19.84449357,
       19.84449357, 19.84449357, 19.84449357, 19.84449357, 19.84449357,
       19.84449357, 19.84449357, 19.84449357, 19.84449357, 19.84449357,
       19.84449357, 19.84449357, 19.84449357, 19.84449357, 19.84449357,
       19.84449357, 19.84449357, 19.84449357, 19.84449357, 19.84

In [96]:
y_test

array([28.875     , 32.5       , 24.75      , 36.5       , 31.        ,
       32.75862069, 25.        , 32.85714286, 14.5       ,  9.57142857,
       16.125     , 35.66666667, 19.18181818, 26.        , 27.625     ,
       29.625     , 11.35714286, 12.875     , 26.71428571, 11.39534884,
       34.75      , 17.625     , 30.63636364, 28.        , 12.375     ,
       32.25      , 18.81818182, 27.75      , 15.66666667, 38.25      ,
       27.125     , 22.71428571, 11.75      , 11.625     , 26.875     ,
       21.82608696, 30.5       , 32.30769231, 19.88888889, 25.15093453,
       22.625     , 34.125     , 32.        , 28.5       , 20.5       ,
       31.5       , 38.79166667, 33.43478261, 26.32608696, 30.25      ,
       18.63636364, 31.75      , 34.75      , 19.54166667, 28.04545455,
       13.86363636, 16.28571429, 32.29411765, 32.625     , 37.43478261,
       30.78571429, 34.95555556, 25.75      , 20.02380952, 27.375     ,
       28.95833333, 33.5       , 20.        , 30.61904762, 34.28

In [97]:
# Accuracy
from sklearn.metrics import mean_squared_error, mean_absolute_error
mse=mean_squared_error(y_test,y_pred)
mae=mean_absolute_error(y_test,y_pred)
print(f'{mse} {mae}')

86.37231440989204 8.085748486264313
